import subprocess

# Lista de paquetes a instalar
paquetes = [
    'asttokens==2.4.1',
    'colorama==0.4.6',
    'comm==0.2.1',
    'debugpy==1.8.1',
    'decorator==5.1.1',
    'executing==2.0.1',
    'ipykernel==6.29.2',
    'ipython==8.21.0',
    'jedi==0.19.1',
    'jupyter_client==8.6.0',
    'jupyter_core==5.7.1',
    'matplotlib-inline==0.1.6',
    'nest-asyncio==1.6.0',
    'packaging==23.2',
    'parso==0.8.3',
    'platformdirs==4.2.0',
    'polars==0.20.10',
    'prompt-toolkit==3.0.43',
    'psutil==5.9.8',
    'psycopg2==2.9.9',
    'pure-eval==0.2.2',
    'Pygments==2.17.2',
    'python-dateutil==2.8.2',
    'python-dotenv==1.0.1',
    'pywin32==306',
    'pyzmq==25.1.2',
    'six==1.16.0',
    'stack-data==0.6.3',
    'tornado==6.4',
    'traitlets==5.14.1',
    'wcwidth==0.2.13',
    'xlsx2csv==0.8.2'
]

# Comando de instalación para cada paquete
for paquete in paquetes:
    subprocess.run(['pip', 'install', paquete])

In [1]:
from Scripts.Transform import CreateDimension, SplitByDelimiter, RenameColumns, CreateFactTable
from Scripts.Extract import ImportDataFromExcel, ImportDataFromPostgreSQL
from Scripts.Load import CrearTablas, InsertDimToPostgreSQL, InsertFactToPostgreSQL

In [2]:
# Variables que usaremos
filepath        = './Atenciones.xlsx'
nombre_columnas = {
                    'Numero Ticket': 'TicketID',
                    'Estado Ticket': 'Estado',
                    'Tipo Atencion': 'TipoAtencion',
                    'Fecha Creacion': 'FechaCreacion',
                    'Fecha Cierre': 'FechaCierre',
                    'Tipo': 'TipoItem',
                    'Tipo Proveedor': 'TipoProveedor'
                }

In [3]:
# 1. Extract
Atenciones = ImportDataFromExcel(filepath = filepath, sheet = 'Hoja1')

In [4]:
# 2. Renombrando
Atenciones = RenameColumns(Atenciones, nombre_columnas)
Atenciones = SplitByDelimiter(Atenciones, 'Ubicacion')
Atenciones = RenameColumns(Atenciones, {'Columna1': 'AgenciaID', 'Columna2': 'Agencia'})
Atenciones = Atenciones.sort('FechaCreacion', descending = False) # Ordenar por FechaCreacion

In [5]:
# 3. Creando los DataFrames
Agencias     = CreateDimension(Atenciones, 'AgenciaID', 'Agencia', 'Direccion', 'Distrito', 'Provincia', 'Departamento')
Estado       = CreateDimension(Atenciones, 'Estado')
TipoAtencion = CreateDimension(Atenciones, 'TipoAtencion')
Proveedor    = CreateDimension(Atenciones, 'Proveedor', 'TipoProveedor')
Item         = CreateDimension(Atenciones, 'ItemID',    'Item',    'TipoItem',  'Categoria')

In [6]:
# 4. Probando Conexion
CrearTablas()
InsertDimToPostgreSQL(Agencias,     'dimAgencia')
InsertDimToPostgreSQL(Item,         'dimItem')
InsertDimToPostgreSQL(Proveedor,    'dimProveedor')
InsertDimToPostgreSQL(TipoAtencion, 'dimTipoAtencion')
InsertDimToPostgreSQL(Estado,       'dimEstado')

Tabla eliminada correctamente.
Tabla eliminada correctamente.
Tabla eliminada correctamente.
Tabla eliminada correctamente.
Tabla eliminada correctamente.
Tabla eliminada correctamente.
Tabla creada correctamente.
Tabla creada correctamente.
Tabla creada correctamente.
Tabla creada correctamente.
Tabla creada correctamente.
Tabla creada correctamente.
Todas las tablas han sido creadas exitosamente.
Datos insertados correctamente.
Datos insertados correctamente.
Datos insertados correctamente.
Datos insertados correctamente.
Datos insertados correctamente.


In [7]:
# Extraer dimensiones con clave subrogada
# 5. Llamando al código
Proveedor    = ImportDataFromPostgreSQL('dimProveedor',    ['ProveedorID',    'Proveedor'])
TipoAtencion = ImportDataFromPostgreSQL('dimTipoAtencion', ['TipoAtencionID', 'TipoAtencion'])
Estado       = ImportDataFromPostgreSQL('dimEstado',       ['EstadoID',       'Estado'])

In [8]:
# 6. Haciendo el Join
Atenciones = CreateFactTable(Atenciones, Proveedor, Estado, TipoAtencion)

In [9]:
# 7. Escribiendo datos
InsertFactToPostgreSQL(Atenciones, 'factAtenciones')

Datos insertados correctamente.


In [10]:
Atenciones

TicketID,EstadoID,TipoAtencionID,FechaCreacion,FechaCierre,AgenciaID,ItemID,ProveedorID
str,i64,i64,date,date,str,i64,i64
"""WO000000409837…",2,3,2022-05-25,2022-06-05,"""255001""",253,13
"""WO000000409878…",2,3,2022-05-25,2022-05-27,"""191054""",43,13
"""WO000000409847…",5,2,2022-05-25,2022-06-11,"""191061""",160,8
"""WO000000409862…",5,2,2022-05-25,2022-05-29,"""191084""",160,8
"""WO000000409850…",5,2,2022-05-25,2022-05-27,"""191141""",160,8
"""WO000000409861…",5,2,2022-05-25,2022-05-27,"""385000""",160,8
"""WO000000409860…",5,2,2022-05-25,2022-05-28,"""192081""",160,8
"""WO000000409850…",5,2,2022-05-25,2022-05-31,"""193005""",160,8
"""WO000000409840…",5,2,2022-05-25,2022-05-28,"""192081""",160,8
